# Описание проекта
Вы работаете в интернет-магазине. Ваша задача проанализировать юнит-экономику и помочь маркетологам разобраться — снижать или увеличивать расходы на маркетинг. У вас есть данные о продажах и расходах. Вы также знаете, что маржинальность магазина равна 40%.

## Задача
1. Посчитать количество уникальных покупателей в каждой когорте;
2. Создать когортный отчёт, добавить в него столбец с возрастом каждой когорты, суммарной валовой прибылью когорты в каждый месяц, посчитайте LTV;
3. Посчитать CAC и ROMI;
4. Посчитать LTV средней когорты.

In [8]:
import pandas as pd
import numpy as np

In [10]:
orders = pd.read_csv('ltv_orders_2.csv')
costs = pd.read_csv('ltv_costs_2.csv')

orders['order_date'] = pd.to_datetime(orders['order_date'])
costs['date'] = pd.to_datetime(costs['date'])

orders['order_month'] = orders['order_date'].astype('datetime64[M]')
costs['month'] = costs['date'].astype('datetime64[M]')


first_orders = orders.groupby('uid').agg({'order_month': 'min'}).reset_index()
first_orders.columns = ['uid', 'first_order_month']

cohort_sizes = (
    first_orders.groupby('first_order_month')
    .agg({'uid': 'nunique'})
    .reset_index()
)
cohort_sizes.columns = ['first_order_month', 'n_buyers']
print(cohort_sizes.head(5))



  first_order_month  n_buyers
0        2019-01-01        46
1        2019-02-01       122
2        2019-03-01       182
3        2019-04-01       173
4        2019-05-01       182


**Вывод:**    
да, в январе не слишком много новых покупателей. Видимо, потратили все деньги на подарки ещё в конце декабря.

In [11]:
margin_rate = 0.4

orders_first_month = pd.merge(orders, first_orders, on='uid')
cohorts = (
    orders_first_month.groupby(['first_order_month', 'order_month'])
    .agg({'revenue': 'sum'})
    .reset_index()
)
report = pd.merge(cohort_sizes, cohorts, on='first_order_month')

report['gp'] = report['revenue'] * margin_rate
report['age'] = (
    report['order_month'] - report['first_order_month']
) / np.timedelta64(1, 'M')
report['age'] = report['age'].round().astype('int')

report['ltv'] = report['gp'] / report['n_buyers']

result = report.pivot_table(
    index='first_order_month', columns='age', values='ltv', aggfunc='mean'
).round()
result

age,0,1,2,3,4,5,6
first_order_month,,,,,,,
2019-01-01,536.0,487.0,567.0,484.0,325.0,70.0,5.0
2019-02-01,551.0,484.0,523.0,442.0,258.0,88.0,17.0
2019-03-01,625.0,493.0,549.0,425.0,247.0,89.0,4.0
2019-04-01,572.0,527.0,456.0,418.0,199.0,20.0,NaN
2019-05-01,538.0,507.0,450.0,260.0,31.0,NaN,NaN
2019-06-01,622.0,496.0,370.0,53.0,NaN,NaN,NaN
2019-07-01,535.0,514.0,121.0,NaN,NaN,NaN,NaN
2019-08-01,580.0,116.0,NaN,NaN,NaN,NaN,NaN
2019-09-01,381.0,NaN,NaN,NaN,NaN,NaN,NaN


**Вывод:**     
получили когортный отчёт и посчитали LTV. Осталось добавить данные о расходах и найти ROMI.

In [14]:
monthly_costs = costs.groupby('month').sum()
report_new = pd.merge(
    report, monthly_costs, left_on='first_order_month', right_on='month'
)
report_new['cac'] = report_new['costs'] / report_new['n_buyers']

report_new['romi'] = report_new['ltv'] / report_new['cac']
output = report_new.pivot_table(
    index='first_order_month', columns='age', values='romi', aggfunc='mean'
)

final_result=report_new.pivot_table(
    index='first_order_month',
    columns='age',
    values='ltv',
    aggfunc='mean')
print(output.cumsum(axis=1).round(2).fillna(''))

age                   0     1     2     3     4     5     6
first_order_month                                          
2019-01-01         0.19  0.36  0.56  0.73  0.84  0.86  0.87
2019-02-01         0.18  0.35  0.52  0.67  0.75  0.78  0.79
2019-03-01         0.20  0.35  0.53  0.66  0.74  0.77  0.77
2019-04-01         0.20  0.38  0.54  0.69  0.76  0.76      
2019-05-01         0.18  0.35  0.51   0.6  0.61            
2019-06-01         0.21  0.37   0.5  0.51                  
2019-07-01         0.18  0.35  0.39                        
2019-08-01         0.20  0.23                              
2019-09-01         0.12                                    


**Вывод:**     
посчитали ROMI магазина. К сожалению, судя по по отчёту, до сих пор ни одна когорта не окупилась. Кажется, маркетологам надо что-то менять.

In [15]:
m6_cum_ltv = final_result.cumsum(axis=1).mean(axis=0)[5]

print('Средний LTV за 6 месяцев после первой покупки:', m6_cum_ltv)

Средний LTV за 6 месяцев после первой покупки: 2358.917911699909


**Вывод:**    
чтобы вложения в маркетинг окупались, привлечение одного покупателя не должно превышать  2358.9 рублей. Полезная информация, маркетологи вам очень благодарны!